### RGMA GPM + feature labels product
- some example figures demosntrating the overlapping rainfall from multiple features
- associated statistics

In [1]:
import os
import sys
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from pathlib import Path

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
def coordinates_processors(data):
    """
    converting longitude/latitude into lon/lat
    data: xarray.dataset coordinated horizontally in lat/lon
    """

    coord_names = []
    for coord_name in data.coords:
        coord_names.append(coord_name)

    if (set(coord_names) & set(['lon','lat'])): # if coordinates set this way...

        data2 = data.rename({'lat': 'latitude'})
        data2 = data2.rename({'lon': 'longitude'})

    else:
        data2 = data

    # check if lon from -180
    if data2.longitude[0] != 0: # -180 to 180

        lon_reset = data2.longitude
        lon_reset = lon_reset.where(lon_reset > 0, 360+lon_reset) # converting lon as 0 to 359.75
        data2.coords['longitude'] = lon_reset # converting lon as -180 to 180
        data2= data2.sortby('longitude')

    # check if latitutde is decreasing
    if (data2.latitude[1] - data2.latitude[0]) < 0:
        data2 = data2.isel(latitude=slice(None, None, -1)) # flipping latitude accoordingly

    return data2

In [4]:
# declare directories
RGMA_DIR = Path('/neelin2020/RGMA_feature_mask/')
GPM_DIR = Path('/neelin2020/RGMA_feature_mask/GPM_ncfiles_2017/')
DATA_DIR = RGMA_DIR / 'data_product/2017/'
AR_DIR = DATA_DIR / 'AR'
MCS_DIR = DATA_DIR / 'MCS'
LPS_DIR = DATA_DIR / 'LPS'
Front_DIR = DATA_DIR / 'Front'
OUT_DIR = DATA_DIR / 'MERGED_FP'

lat_range = [-60, 60] # MSC only tracked within 60 deg.

In [20]:
for mon in range(1,2):

    # load feature datasets
    files = list(AR_DIR.glob('*_{}_*'.format(str(mon).zfill(2))))[0]
    data_AR = xr.open_dataset(files)
    data_AR = coordinates_processors(data_AR)
    data_AR = data_AR.sel(latitude=slice(lat_range[0], lat_range[1]))

    files = list(Front_DIR.glob('Front_cold*_{}_*'.format(str(mon).zfill(2))))[0]
    data_FT_c = xr.open_dataset(files)
    data_FT_c = coordinates_processors(data_FT_c)
    data_FT_c = data_FT_c.sel(latitude=slice(lat_range[0], lat_range[1]))

In [ ]:
# load feature datasets
data_AR = xr.open_dataset(RGMA_DIR / 'AR_ERA5feature_mask_2017_hrly.nc')
data_AR = coordinates_processors(data_AR)
data_AR = data_AR.sel(latitude=slice(lat_range[0], lat_range[1]))

data_FT_c = xr.open_dataset(RGMA_DIR / 'Front_cold_ERA5feature_mask_2017_6hrly.nc')
data_FT_c = coordinates_processors(data_FT_c)
data_FT_c = data_FT_c.sel(latitude=slice(lat_range[0], lat_range[1]))

data_FT_w = xr.open_dataset(RGMA_DIR / 'Front_warm_ERA5feature_mask_2017_6hrly.nc')
data_FT_w = coordinates_processors(data_FT_w)
data_FT_w = data_FT_w.sel(latitude=slice(lat_range[0], lat_range[1]))

data_FT_s = xr.open_dataset(RGMA_DIR / 'Front_stat_ERA5feature_mask_2017_6hrly.nc')
data_FT_s = coordinates_processors(data_FT_s)
data_FT_s = data_FT_s.sel(latitude=slice(lat_range[0], lat_range[1]))

data_LPS = xr.open_dataset(RGMA_DIR / 'LPS_ERA5feature_mask_2017_hrly.nc')
data_LPS = coordinates_processors(data_LPS)
data_LPS = data_LPS.sel(latitude=slice(lat_range[0], lat_range[1]))

data_MCS = xr.open_dataset(RGMA_DIR / 'MCS_ERA5feature_mask_2017_hrly.nc')
data_MCS = coordinates_processors(data_MCS)
data_MCS = data_MCS.sel(latitude=slice(lat_range[0], lat_range[1]))

file_list = sorted(list(GPM_DIR.glob('GPM_IMERGE_V06_201701*')))
data_GPM = xr.open_mfdataset(file_list)
data_GPM = coordinates_processors(data_GPM)
data_GPM_sub = data_GPM.sel(time=slice(datetime(2017,1,1,0),datetime(2017,12,31,23)),
                            latitude=slice(lat_range[0], lat_range[1]))

In [ ]:
# reset variable name as feature_tag (ar_tag, mcs_tag, lps_tag, front_tg)
data_AR = data_AR.rename({'ar_binary_tag': 'ar_tag'})
data_FT_c = data_FT_c.rename({'fronts': 'front_c_tag'})
data_FT_w = data_FT_w.rename({'fronts': 'front_w_tag'})
data_FT_s = data_FT_s.rename({'fronts': 'front_s_tag'})
data_LPS = data_LPS.rename({'feature_mask': 'lps_tag'})
data_MCS = data_MCS.rename({'feature_mask': 'mcs_tag'})

### Extra modification for MCS and LPS

In [ ]:
lon_era5 = data_AR.longitude
lat_era5 = data_AR.latitude

In [ ]:
# MCS is defined at xx:30 based on MERGE-IR, we assume small changes in the feature within 
# the 30-min time frame for simplicity

MCS_array = np.zeros(data_AR.ar_tag.shape)
MCS_array[:len(data_MCS.mcs_tag.time),:,:] = data_MCS.mcs_tag.values # fill the new array by the original tag

data_MCS_re = xr.Dataset(data_vars=dict(mcs_tag=(['time','latitude','longitude'], MCS_array)),
                         coords=dict(time=data_AR.time.values,
                                     latitude=(['latitude'], data_MCS.latitude.values),
                                     longitude=(['longitude'], data_MCS.longitude.values)))
del MCS_array

In [ ]:
# # LPS feature is provided with the feature center, now we simply use a 5-deg box mask to 
# # present the feature associated boundaries

# LPS_array = np.zeros(data_LPS.lps_tag.shape)
# range_sel = int(5/0.25) # 5-deg box

# for t in range(len(data_LPS.time)):
    
#     tmp = data_LPS.isel(time=t).lps_tag.values
#     (idy_list, idx_list) = np.where(tmp == 1) # find index of LPS
    
#     for (idy, idx) in zip(idy_list, idx_list):
        
#         LPS_array[t,(idy-range_sel):(idy+range_sel+1),(idx-range_sel):(idx+range_sel+1)] = 1
        
# data_LPS_re = xr.Dataset(data_vars=dict(lps_tag=(['time','latitude','longitude'], LPS_array)),
#                          coords=dict(time=data_LPS.time.values,
#                                      latitude=(['latitude'], data_LPS.latitude.values),
#                                      longitude=(['longitude'], data_LPS.longitude.values)))
# del LPS_array

In [ ]:
# interpolate GPM into ERA5 grid using linear fitting
data_GPM_intp = data_GPM_sub.interp(longitude=lon_era5, latitude=lat_era5, method='linear').fillna(0)

In [ ]:
data_GPM_intp = data_GPM_intp.transpose('time','latitude','longitude') # transpose data coordinate

In [ ]:
%%time
# final step: merge into a feature-precip (FP) xarray dataset
# note: resample 6H due to front data
data_FP_merged = xr.merge([data_GPM_intp.resample(time='6H').nearest(),
                           data_AR.resample(time='6H').nearest(),
                           data_FT_c.resample(time='6H').nearest(),
                           data_FT_w.resample(time='6H').nearest(),
                           data_FT_s.resample(time='6H').nearest(),
                           data_LPS.resample(time='6H').nearest(),
                           data_MCS_re.resample(time='6H').nearest()])
data_FP_merged['precipitationCal'] = data_FP_merged['precipitationCal'].where(data_FP_merged['precipitationCal'] > 0, 0)

In [ ]:
# save tags as int8 for data compression
data_FP_merged = data_FP_merged.fillna(0)
data_FP_merged['ar_tag'].values = data_FP_merged.ar_tag.values.astype('byte')
data_FP_merged['front_c_tag'].values = data_FP_merged.front_c_tag.values.astype('byte')
data_FP_merged['front_w_tag'].values = data_FP_merged.front_w_tag.values.astype('byte')
data_FP_merged['front_s_tag'].values = data_FP_merged.front_s_tag.values.astype('byte')
data_FP_merged['lps_tag'].values = data_FP_merged.lps_tag.values.astype('byte')
data_FP_merged['mcs_tag'].values = data_FP_merged.mcs_tag.values.astype('byte')
# reassign time
data_FP_merged.coords['time'] = data_AR.resample(time='6H').nearest().time.values

In [ ]:
# save netcdf file by month
date_range = pd.date_range(data_FP_merged.time[0].values, data_FP_merged.time[-1], freq='6H')

for mon in range(1,13): # slice the full data into monthly data
    idx = np.where(date_range.month == mon) # extract specific month
    data_FP_month = data_FP_merged.isel(time=idx)
    data_FP_merged.to_netcdf(OUT_DIR / 'GPM_feature_merged_{}_{}.nc'.format(date_range[0].year, str(mon).zfill(2)))
    print('GPM_feature_merged_{}_{}.nc saved...'.format(date_range[0].year, str(mon).zfill(2)))

### Stats from here: some test for feature-associated extractioon 



In [ ]:
# looks like MCS + unexplained over 1, but its not
(Unexp_contr+MCS_contr).max()

### some statistics from the precipitation contribution